In [1]:
import os
import sys
import time
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from scipy.stats import linregress
from sklearn.neighbors import KernelDensity
from datetime import date, datetime, timedelta

## Read Data

In [2]:
## Read data

# Variables - change these as needed
job_id = '02Feb2021'
week_minus_one = '24-Jan-2021'

# Fixed variables
dc_location_allowed = {'GB40','GB47','GB73'}
test_weeks = 8
horizon_weeks = 12
data_path = '/Users/sandeepdhankhar/OneDrive - Aera Technology/8 Project Mars/Data/'

dc_forecast = pd.read_csv(os.path.join(data_path,job_id,'input','dc_forecast.csv'), dtype = {'dfu':str,'location':str})
dc_actuals = pd.read_csv(os.path.join(data_path,job_id,'input','dc_actuals.csv'), dtype = {'dfu':str,'location':str})
prodn_actuals = pd.read_csv(os.path.join(data_path,job_id,'input','production_actuals.csv'), dtype = {'dfu':str,'location':str})    


## Functions

In [3]:
## Functions
def get_year_week(dateSeries):
    yearSeries = dateSeries.apply(lambda x: x.strftime("%Y")).astype(str)
    weekSeries = dateSeries.apply(lambda x: x.strftime("%U")).astype(str)  #week starting Sunday - same as Mars
    yearWeek = yearSeries + weekSeries
    return yearWeek.astype(int)

def get_missing_snapshots(single_grain,dfu_name,location_name,suffix):
    # Snapshots present
    snapshotsPresent = set(single_grain['Week_Start_Date'])
    numOfSnapshots = len(snapshotsPresent)
    firstSnapshotDate = min(snapshotsPresent)
    lastSnapshotDate = max(snapshotsPresent)
    # Find missing snapshots date
    all_snapshots = set(pd.date_range(start = firstSnapshotDate, end = lastSnapshotDate, freq = 'W'))
    missingSnapshots = all_snapshots - snapshotsPresent
    missingSnapshots = [x.strftime('%Y-%m-%d') for x in missingSnapshots]
    # Consolidate
    summary = {'dfu':dfu_name,'location':location_name,'NumOfSnapshot'+suffix:numOfSnapshots,
               'FirstSnapshot'+suffix:firstSnapshotDate.strftime('%Y-%m-%d'),
               'LastSnapshot'+suffix:lastSnapshotDate.strftime('%Y-%m-%d'),
               'MissingSnapshot'+suffix:missingSnapshots,
               'NumberOfSnapshotsMissing'+suffix:len(missingSnapshots)
               }
    return summary

def get_snapshot_summary(data,group,suffix):
    grain_summary = []
    grain_groups = data.groupby(group)
    for name,group in grain_groups:
        grain_summary.append(get_missing_snapshots(group,name[0],name[1],suffix))
    data_summary = pd.DataFrame(grain_summary)    
    #data_summary = data_summary[data_summary['NumberOfSnapshotsMissing'] != 0]
    return data_summary

def insert_missing_time_periods_prodn(prodn):
    # looping over each grain
    grain_groups = prodn.groupby(['dfu','location','line'])
    grain_complete = []
    for name,singleGrain in grain_groups:
        # Re-index - add all time periods between start and end date
        singleGrain = singleGrain.set_index('Week_Start_Date')
        idx = pd.date_range(start = min(singleGrain.index),end = max(singleGrain.index),freq='W')
        singleGrain = singleGrain.reindex(list(idx))
        # fill na
        singleGrain[['dfu','location','line']] = singleGrain[['dfu','location','line']].fillna(method='ffill')
        singleGrain[['actual_production_in_ea','scheduled_production_in_ea']] = singleGrain[['actual_production_in_ea','scheduled_production_in_ea']].fillna(0)
        singleGrain[['csl','ats']] = singleGrain[['csl','ats']].fillna(100)
        grain_complete.append(singleGrain)
    # re-arrange cols
    prodn_complete = pd.concat(grain_complete).reset_index()
    return prodn_complete[['dfu','location','line','Week_Start_Date','actual_production_in_ea','scheduled_production_in_ea','csl','ats']]

def mad_outlier_treatment(data,col):
    d = data.copy()
    # 2MAD method
    d['ShiftedQty'] = d[col].shift(1)
    d['RollingMedian'] = d['ShiftedQty'].transform(lambda x: x.rolling(6, 1).median())
    d['AbsDeviation'] = abs(d['RollingMedian'] - d[col])
    d['MAD'] = d['AbsDeviation'].median()
    d['OutlierFlag_2MAD'] = [1 if dev > 2*mad else 0 for dev,mad in zip(d['AbsDeviation'],d['MAD'])]
    d[col+'_MAD'] = [z if x==1 else y for x,y,z in zip(d['OutlierFlag_2MAD'],d[col],d['RollingMedian'])]
    d = d.drop(['ShiftedQty','RollingMedian','AbsDeviation','MAD','OutlierFlag_2MAD'],axis=1) 
    return d

def iqr_outlier_treatment(data,col):
    d = data.copy()
    q1,q3 = np.percentile(d[col],[25,75])
    IQR = q3-q1
    lower_bound = q1 - (IQR*1.5)
    upper_bound = q3 + (IQR*1.5)
    d[col+'_IQR'] = d[col]
    d[col+'_IQR'] = [lower_bound if x < lower_bound else x for x in d[col]]
    d[col+'_IQR'] = [upper_bound if x > upper_bound else x for x in d[col]]
    return d

def get_ats_csl_correlation(data,ats_col):
    ats = data[ats_col]
    csl = data['csl']
    csl = csl[~ats.isin([np.nan, np.inf, -np.inf])] # Remove unwanted values
    ats = ats[~ats.isin([np.nan, np.inf, -np.inf])] # Remove unwanted values
    out = linregress(ats,csl)
    return pd.DataFrame({'Correlation':[out.rvalue],'P-Value':[out.pvalue],'NumOfPoints':[len(ats)],'Coefficient':[out.slope]})

def recalculate_ats(row):
    if (row['actual_production_in_ea'] == 0) & (row['scheduled_production_in_ea'] == 0):
        return 100
    elif row['scheduled_production_in_ea'] == 0:
        return 0
    else:
        return row['actual_production_in_ea']*100/row['scheduled_production_in_ea']
    
def cap_ats(ats):
    if ats > 120:
        return 120
    elif ats < 0:
        return 0
    else:
        return ats
    
def get_max_density_outcome(series):
    #sns.distplot(data, hist=False)
    #plt.show()
    X = series.to_numpy().reshape(-1,1)
    kde = KernelDensity(kernel='gaussian', bandwidth=1).fit(X)
    X_plot = np.linspace(-100, 300, 1000)[:, np.newaxis]    
    log_dens = np.exp(kde.score_samples(X_plot))
    max_density_val = X_plot[np.argmax(log_dens),0]
    return max_density_val

def train_linear_model(data,y,x):
    slope, intercept, r_value, p_value, std_err = linregress(data[x], data[y])
    return pd.DataFrame({'slope':[slope], 'intercept':[intercept]})

## Data cleaning

In [4]:
# Column rename
dc_forecast.rename(columns = {'forecast_week':'Week_Start_Date'}, inplace = True)
dc_actuals.rename(columns = {'week':'Week_Start_Date'}, inplace = True)
prodn_actuals.rename(columns = {'date':'Week_Start_Date'}, inplace = True)

# Convert to datetime
dc_forecast['Week_Start_Date'] = pd.to_datetime(dc_forecast['Week_Start_Date'],format='%Y-%m-%d')
dc_actuals['Week_Start_Date'] = pd.to_datetime(dc_actuals['Week_Start_Date'],format='%Y-%m-%d')
prodn_actuals['Week_Start_Date'] = pd.to_datetime(prodn_actuals['Week_Start_Date'],format='%Y-%m-%d')

# Filter unknown data
dc_forecast = dc_forecast[~((dc_forecast['dfu'] == "Not Set") | (dc_forecast['location'] == "not_available"))]
dc_actuals = dc_actuals[~((dc_actuals['dfu'] == "Not Set") | (dc_actuals['location'] == "not_available"))]
prodn_actuals = prodn_actuals[~((prodn_actuals['dfu'] == "Not Set") | (prodn_actuals['location'] == "not_available") | (prodn_actuals['line'] == "Not Set"))]
dc_actuals = dc_actuals[dc_actuals['location'].isin(dc_location_allowed)]

# Keep historical data
week_minus_one = datetime.strptime(week_minus_one,'%d-%b-%Y')
dc_forecast = dc_forecast[dc_forecast['Week_Start_Date'] <= week_minus_one]
dc_actuals = dc_actuals[dc_actuals['Week_Start_Date'] <= week_minus_one]
prodn_actuals = prodn_actuals[prodn_actuals['Week_Start_Date'] <= week_minus_one]

# Get top DFU's - 90% portfolio share in past 1 year
date_now = datetime.now()
date_year_ago = date_now - timedelta(days = 365)
cutOff = 90
lastOneYearActual = dc_actuals.loc[dc_actuals['Week_Start_Date'] >= date_year_ago]
aggLastOneYear = lastOneYearActual.groupby(['dfu','location']).agg(PercentageShare=('actuals',lambda x: round(x.sum()*100/sum(lastOneYearActual['actuals']),2))).reset_index()
aggLastOneYear = aggLastOneYear.sort_values(by=['PercentageShare'], ascending=False).reset_index(drop=True)
aggLastOneYear['CumulativeShare'] = aggLastOneYear['PercentageShare'].cumsum()    
aggLastOneYear = aggLastOneYear[aggLastOneYear['CumulativeShare'] < cutOff]
aggLastOneYear.head()


dfu location  PercentageShare  CumulativeShare
0  376308     GB47             9.98             9.98
1  376308     GB73             7.80            17.78
2  376308     GB40             2.77            20.55
3  413506     GB47             0.67            21.22
4  375129     GB47             0.57            21.79

In [5]:
#aggLastOneYear.to_csv(os.path.join(data_path,job_id,'exp','volume_share.csv'),index=False)


In [6]:
# Keep top DFU's
top_dfu = aggLastOneYear.dfu.unique()
dc_forecast  = dc_forecast[dc_forecast['dfu'].isin(top_dfu)]
dc_actuals  = dc_actuals[dc_actuals['dfu'].isin(top_dfu)]
#prodn_actuals  = prodn_actuals[prodn_actuals['dfu'].isin(top_dfu)]

# Remove data before 1st Jan 2020 => ats,csl missing in most of the grains
prodn_actuals_complete = prodn_actuals[prodn_actuals['Week_Start_Date'] >= datetime(2020, 1, 1)]

## ATS - CSL correlation

In [7]:
prodn_actuals_complete = prodn_actuals_complete.sort_values(by=['dfu','location','Week_Start_Date'])
prodn_actuals_complete['ats'] = prodn_actuals_complete['actual_production_in_ea']*100/prodn_actuals_complete['scheduled_production_in_ea']
prodn_actuals_complete = prodn_actuals_complete.replace(np.inf,0)
prodn_actuals_complete['ats'] = prodn_actuals_complete.groupby(['dfu','location'])['ats'].fillna(method = 'ffill')
prodn_actuals_complete['ats'] = prodn_actuals_complete.groupby(['dfu','location'])['ats'].fillna(method = 'bfill')
prodn_actuals_complete.head()

Week_Start_Date        ats     dfu           line location  \
17      2020-01-05  98.004900  101559  SLO MALTESERS     GB18   
18      2020-01-12   0.000000  101559  SLO MALTESERS     GB18   
19      2020-01-19   2.738386  101559  SLO MALTESERS     GB18   
20      2020-01-26   0.000000  101559  SLO MALTESERS     GB18   
21      2020-02-02   0.000000  101559  SLO MALTESERS     GB18   

    actual_production_in_ea  csl  scheduled_production_in_ea  
17                   2800.0  0.0                      2857.0  
18                      0.0  0.0                      2621.0  
19                     56.0  0.0                      2045.0  
20                      0.0  0.0                      1860.0  
21                      0.0  0.0                       723.0

In [8]:
prodn_actuals_complete[['dfu','location']].drop_duplicates().shape

(332, 2)

In [9]:
# Calculate lag1 and lag2
prodn_actuals_complete['ats_lag1'] = prodn_actuals_complete.groupby(['dfu','location'])['ats'].shift(1)
prodn_actuals_complete['ats_lag2'] = prodn_actuals_complete.groupby(['dfu','location'])['ats'].shift(2)
prodn_actuals_complete['ats_lag3'] = prodn_actuals_complete.groupby(['dfu','location'])['ats'].shift(3)
prodn_actuals_complete['ats_lag4'] = prodn_actuals_complete.groupby(['dfu','location'])['ats'].shift(4)
prodn_actuals_complete['ats_lag5'] = prodn_actuals_complete.groupby(['dfu','location'])['ats'].shift(5)

# Correlation
ats_csl_corr_lag0 = prodn_actuals_complete.groupby(['dfu','location']).apply(get_ats_csl_correlation,'ats').reset_index().sort_values('P-Value')
ats_csl_corr_lag1 = prodn_actuals_complete.groupby(['dfu','location']).apply(get_ats_csl_correlation,'ats_lag1').reset_index().sort_values('P-Value')
ats_csl_corr_lag2 = prodn_actuals_complete.groupby(['dfu','location']).apply(get_ats_csl_correlation,'ats_lag2').reset_index().sort_values('P-Value')
ats_csl_corr_lag3 = prodn_actuals_complete.groupby(['dfu','location']).apply(get_ats_csl_correlation,'ats_lag3').reset_index().sort_values('P-Value')
ats_csl_corr_lag4 = prodn_actuals_complete.groupby(['dfu','location']).apply(get_ats_csl_correlation,'ats_lag4').reset_index().sort_values('P-Value')
ats_csl_corr_lag5 = prodn_actuals_complete.groupby(['dfu','location']).apply(get_ats_csl_correlation,'ats_lag5').reset_index().sort_values('P-Value')

#prodn_actuals_complete.to_csv(os.path.join(data_path,job_id,'exp','prodn_actuals_complete.csv'),index=False)
#ats_csl_corr_lag2.to_csv(os.path.join(data_path,job_id,'exp','ats_csl_corr_lag2.csv'),index=False)


In [14]:
# Combine all results
ats_csl_corr_lag0['lag'] = 0
ats_csl_corr_lag1['lag'] = 1
ats_csl_corr_lag2['lag'] = 2
ats_csl_corr_lag3['lag'] = 3
ats_csl_corr_lag4['lag'] = 4
ats_csl_corr_lag5['lag'] = 5

ats_csl_corr_with_p_value = pd.concat([ats_csl_corr_lag0,ats_csl_corr_lag1,ats_csl_corr_lag2,ats_csl_corr_lag3,ats_csl_corr_lag4,ats_csl_corr_lag5])
#ats_csl_corr_with_p_value.to_csv(os.path.join(data_path,job_id,'exp','ats_csl_corr_with_p_value.csv'),index=False)


In [10]:
# Examples where correlation is significant
lag0_significant = ats_csl_corr_lag0[ats_csl_corr_lag0['Correlation'] >= 0]
lag0_significant = lag0_significant[lag0_significant['P-Value']< 0.05]
print('Number of grains with signifiant correlations at lag0 - {}'.format(lag0_significant.shape[0]))

lag1_significant = ats_csl_corr_lag1[ats_csl_corr_lag1['Correlation'] >= 0]
lag1_significant = lag1_significant[lag1_significant['P-Value']< 0.05]
print('Number of grains with signifiant correlations at lag1 - {}'.format(lag1_significant.shape[0]))

lag2_significant = ats_csl_corr_lag2[ats_csl_corr_lag2['Correlation'] >= 0]
lag2_significant = lag2_significant[lag2_significant['P-Value']< 0.05]
print('Number of grains with signifiant correlations at lag2 - {}'.format(lag2_significant.shape[0]))

lag3_significant = ats_csl_corr_lag3[ats_csl_corr_lag3['Correlation'] >= 0]
lag3_significant = lag3_significant[lag3_significant['P-Value']< 0.05]
print('Number of grains with signifiant correlations at lag3 - {}'.format(lag3_significant.shape[0]))

lag4_significant = ats_csl_corr_lag4[ats_csl_corr_lag4['Correlation'] >= 0]
lag4_significant = lag4_significant[lag4_significant['P-Value']< 0.05]
print('Number of grains with signifiant correlations at lag4 - {}'.format(lag4_significant.shape[0]))

lag5_significant = ats_csl_corr_lag5[ats_csl_corr_lag5['Correlation'] >= 0]
lag5_significant = lag5_significant[lag5_significant['P-Value']< 0.05]
print('Number of grains with signifiant correlations at lag5 - {}'.format(lag5_significant.shape[0]))





Number of grains with signifiant correlations at lag0 - 20
Number of grains with signifiant correlations at lag1 - 20
Number of grains with signifiant correlations at lag2 - 41
Number of grains with signifiant correlations at lag3 - 37
Number of grains with signifiant correlations at lag4 - 38
Number of grains with signifiant correlations at lag5 - 38


In [11]:
# When ATS is less than 100
prodn_temp = prodn_actuals_complete[prodn_actuals_complete['ats'] < 100]
ats_csl_corr_lag0_temp = prodn_temp.groupby(['dfu','location']).apply(get_ats_csl_correlation,'ats').reset_index().sort_values('P-Value')
count = ats_csl_corr_lag0_temp[ats_csl_corr_lag0_temp['P-Value']< 0.05].shape[0]
print('Number of grains with signifiant correlations when ATS < 100 at lag0 - {}'.format(count))

prodn_temp = prodn_actuals_complete[prodn_actuals_complete['ats_lag1'] < 100]
ats_csl_corr_lag1_temp = prodn_temp.groupby(['dfu','location']).apply(get_ats_csl_correlation,'ats_lag1').reset_index().sort_values('P-Value')
count = ats_csl_corr_lag1_temp[ats_csl_corr_lag1_temp['P-Value']< 0.05].shape[0]
print('Number of grains with signifiant correlations when ATS < 100 at lag1 - {}'.format(count))

prodn_temp = prodn_actuals_complete[prodn_actuals_complete['ats_lag2'] < 100]
ats_csl_corr_lag2_temp = prodn_temp.groupby(['dfu','location']).apply(get_ats_csl_correlation,'ats_lag2').reset_index().sort_values('P-Value')
count = ats_csl_corr_lag2_temp[ats_csl_corr_lag2_temp['P-Value']< 0.05].shape[0]
print('Number of grains with signifiant correlations when ATS < 100 at lag2 - {}'.format(count))

prodn_temp = prodn_actuals_complete[prodn_actuals_complete['ats_lag3'] < 100]
ats_csl_corr_lag3_temp = prodn_temp.groupby(['dfu','location']).apply(get_ats_csl_correlation,'ats_lag3').reset_index().sort_values('P-Value')
count = ats_csl_corr_lag3_temp[ats_csl_corr_lag3_temp['P-Value']< 0.05].shape[0]
print('Number of grains with signifiant correlations when ATS < 100 at lag3 - {}'.format(count))

prodn_temp = prodn_actuals_complete[prodn_actuals_complete['ats_lag4'] < 100]
ats_csl_corr_lag4_temp = prodn_temp.groupby(['dfu','location']).apply(get_ats_csl_correlation,'ats_lag4').reset_index().sort_values('P-Value')
count = ats_csl_corr_lag4_temp[ats_csl_corr_lag4_temp['P-Value']< 0.05].shape[0]
print('Number of grains with signifiant correlations when ATS < 100 at lag4 - {}'.format(count))

prodn_temp = prodn_actuals_complete[prodn_actuals_complete['ats_lag5'] < 100]
ats_csl_corr_lag5_temp = prodn_temp.groupby(['dfu','location']).apply(get_ats_csl_correlation,'ats_lag5').reset_index().sort_values('P-Value')
count = ats_csl_corr_lag5_temp[ats_csl_corr_lag5_temp['P-Value']< 0.05].shape[0]
print('Number of grains with signifiant correlations when ATS < 100 at lag5 - {}'.format(count))


Number of grains with signifiant correlations when ATS < 100 at lag0 - 31
Number of grains with signifiant correlations when ATS < 100 at lag1 - 30
Number of grains with signifiant correlations when ATS < 100 at lag2 - 42
Number of grains with signifiant correlations when ATS < 100 at lag3 - 35
Number of grains with signifiant correlations when ATS < 100 at lag4 - 33
Number of grains with signifiant correlations when ATS < 100 at lag5 - 35


In [14]:
## Merge all lags together
ats_csl_corr_lag0_temp['lag_var'] = 'lag0'
ats_csl_corr_lag1_temp['lag_var'] = 'lag1'
ats_csl_corr_lag2_temp['lag_var'] = 'lag2'
ats_csl_corr_lag3_temp['lag_var'] = 'lag3'
ats_csl_corr_lag4_temp['lag_var'] = 'lag4'
ats_csl_corr_lag5_temp['lag_var'] = 'lag5'
corr_all_lags = pd.concat([ats_csl_corr_lag0_temp,ats_csl_corr_lag1_temp,ats_csl_corr_lag2_temp,ats_csl_corr_lag3_temp,ats_csl_corr_lag4_temp,ats_csl_corr_lag5_temp])

## Find the best correlation
corr_all_lags = corr_all_lags.pivot(index=['dfu','location'],columns=['lag_var'],values='Correlation').reset_index()
c = ['lag0','lag1','lag2','lag3','lag4','lag5']
corr_all_lags = corr_all_lags.assign(Max_Corr = corr_all_lags[c].max(axis=1), Best_Lag=corr_all_lags[c].idxmax(axis=1))

## Add production share
date_now = datetime.now()
date_year_ago = date_now - timedelta(days = 365)
cutOff = 100
lastOneYearActual = prodn_actuals_complete.loc[prodn_actuals_complete['Week_Start_Date'] >= date_year_ago]
aggLastOneYear = lastOneYearActual.groupby(['dfu','location']).agg(PercentageShare=('actual_production_in_ea',lambda x: round(x.sum()*100/sum(lastOneYearActual['actual_production_in_ea']),2))).reset_index()
aggLastOneYear = aggLastOneYear.sort_values(by=['PercentageShare'], ascending=False).reset_index(drop=True)
aggLastOneYear['CumulativeShare'] = aggLastOneYear['PercentageShare'].cumsum()    
aggLastOneYear = aggLastOneYear[aggLastOneYear['CumulativeShare'] < cutOff]
corr_all_lags = corr_all_lags.merge(aggLastOneYear[['dfu','location','PercentageShare']],on=['dfu','location'])
corr_all_lags.head()

dfu location  lag0  lag1  lag2  lag3  lag4  lag5  Max_Corr Best_Lag  \
0  101559     GB18   0.0   0.0   0.0   0.0   0.0   0.0       0.0     lag0   
1  101635     GB18   0.0   0.0   0.0   0.0   0.0   0.0       0.0     lag0   
2  101635     NL01   0.0   0.0   0.0   0.0   0.0   0.0       0.0     lag0   
3  102351     DE04   0.0   0.0   0.0   0.0   0.0   0.0       0.0     lag0   
4  102351     NL01   0.0   0.0   0.0   0.0   0.0   0.0       0.0     lag0   

   PercentageShare  
0             0.04  
1             0.54  
2             2.96  
3             0.12  
4             0.50

In [15]:
corr_all_lags.to_csv(os.path.join(data_path,job_id,'exp','ats_corr_all_lags_ats_less_than_100.csv'),index=False)
#prodn_actuals_complete.to_csv(os.path.join(data_path,job_id,'exp','ats_csl_all_lags_data.csv'),index=False)

In [98]:
## Merge all lags together
ats_csl_corr_lag0['lag_var'] = 'lag0'
ats_csl_corr_lag1['lag_var'] = 'lag1'
ats_csl_corr_lag2['lag_var'] = 'lag2'
ats_csl_corr_lag3['lag_var'] = 'lag3'
ats_csl_corr_lag4['lag_var'] = 'lag4'
ats_csl_corr_lag5['lag_var'] = 'lag5'
corr_all_lags = pd.concat([ats_csl_corr_lag0,ats_csl_corr_lag1,ats_csl_corr_lag2,ats_csl_corr_lag3,ats_csl_corr_lag4,ats_csl_corr_lag5])

## Find the best correlation
corr_all_lags = corr_all_lags.pivot(index=['dfu','location'],columns=['lag_var'],values='Correlation').reset_index()
c = ['lag0','lag1','lag2','lag3','lag4','lag5']
corr_all_lags = corr_all_lags.assign(Max_Corr = corr_all_lags[c].max(axis=1), Best_Lag=corr_all_lags[c].idxmax(axis=1))

## Add production share
date_now = datetime.now()
date_year_ago = date_now - timedelta(days = 365)
cutOff = 100
lastOneYearActual = prodn_actuals_complete.loc[prodn_actuals_complete['Week_Start_Date'] >= date_year_ago]
aggLastOneYear = lastOneYearActual.groupby(['dfu','location']).agg(PercentageShare=('actual_production_in_ea',lambda x: round(x.sum()*100/sum(lastOneYearActual['actual_production_in_ea']),2))).reset_index()
aggLastOneYear = aggLastOneYear.sort_values(by=['PercentageShare'], ascending=False).reset_index(drop=True)
aggLastOneYear['CumulativeShare'] = aggLastOneYear['PercentageShare'].cumsum()    
aggLastOneYear = aggLastOneYear[aggLastOneYear['CumulativeShare'] < cutOff]
corr_all_lags = corr_all_lags.merge(aggLastOneYear[['dfu','location','PercentageShare']],on=['dfu','location'])
corr_all_lags.head()

dfu location  lag0  lag1  lag2  lag3  lag4  lag5  Max_Corr Best_Lag  \
0  101559     GB18   0.0   0.0   0.0   0.0   0.0   0.0       0.0     lag0   
1  101635     GB18   0.0   0.0   0.0   0.0   0.0   0.0       0.0     lag0   
2  101635     NL01   0.0   0.0   0.0   0.0   0.0   0.0       0.0     lag0   
3  102351     DE04   0.0   0.0   0.0   0.0   0.0   0.0       0.0     lag0   
4  102351     NL01   0.0   0.0   0.0   0.0   0.0   0.0       0.0     lag0   

   PercentageShare  
0             0.04  
1             0.54  
2             2.96  
3             0.12  
4             0.50

In [102]:
#corr_all_lags.to_csv(os.path.join(data_path,job_id,'exp','ats_corr_all_lags.csv'),index=False)
#prodn_actuals_complete.to_csv(os.path.join(data_path,job_id,'exp','ats_csl_all_lags_data.csv'),index=False)

## Bias-CSL correlation